In [1]:
import sys
sys.path.append('/home/paperspace/fastai/courses/ml1')

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import re

In [4]:
PATH = '/home/paperspace/kaggle/titanic/data/'

In [6]:
#!ls {PATH}

In [7]:
ex = pd.read_csv(f'{PATH}gender_submission.csv', low_memory=False)

In [13]:
# the submission should look like this
display_all(ex.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417
PassengerId,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227,1228,1229,1230,1231,1232,1233,1234,1235,1236,1237,1238,1239,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
Survived,0,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,1,0,1,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,1,

In [14]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}test.csv', low_memory=False)

In [15]:
# in order to see the data more clearly
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [16]:
display_all(df_raw.tail().T)

,886,887,888,889,890
PassengerId,887,888,889,890,891
Survived,0,1,0,1,0
Pclass,2,1,3,1,3
Name,"Montvila, Rev. Juozas","Graham, Miss. Margaret Edith","Johnston, Miss. Catherine Helen ""Carrie""","Behr, Mr. Karl Howell","Dooley, Mr. Patrick"
Sex,male,female,female,male,male
Age,27,19,NaN,26,32
SibSp,0,0,1,0,0
Parch,0,0,2,0,0
Ticket,211536,112053,W./C. 6607,111369,370376
Fare,13,30,23.45,30,7.75


In [17]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,891,NaN,NaN,NaN,446,257.354,1,223.5,446,668.5,891
Survived,891,NaN,NaN,NaN,0.383838,0.486592,0,0,0,1,1
Pclass,891,NaN,NaN,NaN,2.30864,0.836071,1,2,3,3,3
Name,891,891,"Boulos, Mr. Hanna",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,714,NaN,NaN,NaN,29.6991,14.5265,0.42,20.125,28,38,80
SibSp,891,NaN,NaN,NaN,0.523008,1.10274,0,0,0,1,8
Parch,891,NaN,NaN,NaN,0.381594,0.806057,0,0,0,0,6
Ticket,891,681,CA. 2343,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,891,NaN,NaN,NaN,32.2042,49.6934,0,7.9104,14.4542,31,512.329


In [18]:
# STEP 1: Feature Engineering

In [19]:
# NAME

In [20]:
def get_title(name):
    title_search = re.search('([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

In [21]:
df_raw['Title'] = df_raw['Name'].apply(get_title)
df_test['Title'] = df_test['Name'].apply(get_title)

In [22]:
display_all(df_raw.tail().T)

,886,887,888,889,890
PassengerId,887,888,889,890,891
Survived,0,1,0,1,0
Pclass,2,1,3,1,3
Name,"Montvila, Rev. Juozas","Graham, Miss. Margaret Edith","Johnston, Miss. Catherine Helen ""Carrie""","Behr, Mr. Karl Howell","Dooley, Mr. Patrick"
Sex,male,female,female,male,male
Age,27,19,NaN,26,32
SibSp,0,0,1,0,0
Parch,0,0,2,0,0
Ticket,211536,112053,W./C. 6607,111369,370376
Fare,13,30,23.45,30,7.75


In [23]:
df_raw.Title.value_counts()/len(df_raw)

Mr          0.580247
Miss        0.204265
Mrs         0.140292
Master      0.044893
Dr          0.007856
Rev         0.006734
Mlle        0.002245
Col         0.002245
Major       0.002245
Ms          0.001122
Lady        0.001122
Don         0.001122
Mme         0.001122
Countess    0.001122
Sir         0.001122
Capt        0.001122
Jonkheer    0.001122
Name: Title, dtype: float64

In [24]:
df_raw['Title'] = df_raw['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_raw['Title'] = df_raw['Title'].replace('Mlle', 'Miss')
df_raw['Title'] = df_raw['Title'].replace('Ms', 'Miss')
df_raw['Title'] = df_raw['Title'].replace('Mme', 'Mrs')

In [25]:
df_test['Title'] = df_test['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df_test['Title'] = df_test['Title'].replace('Mlle', 'Miss')
df_test['Title'] = df_test['Title'].replace('Ms', 'Miss')
df_test['Title'] = df_test['Title'].replace('Mme', 'Mrs')

In [26]:
df_test.Title.value_counts()/len(df_test)

Mr        0.574163
Miss      0.188995
Mrs       0.172249
Master    0.050239
Rare      0.014354
Name: Title, dtype: float64

In [27]:
#FamilySize
df_raw['FamilySize'] = df_raw['SibSp'] + df_raw['Parch'] + 1
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

In [28]:
df_raw.FamilySize.count()

891

In [29]:
# FARE
df_raw.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [14]:
df_raw.loc[ df_raw['Fare'] <= 7.91, 'Fare'] = int(0)
df_raw.loc[(df_raw['Fare'] > 7.91) & (df_raw['Fare'] <= 14.454), 'Fare'] = int(1)
df_raw.loc[(df_raw['Fare'] > 14.454) & (df_raw['Fare'] <= 31), 'Fare']   = int(2)
df_raw.loc[ df_raw['Fare'] > 31, 'Fare'] = int(3)
    

In [33]:
df_test.loc[ df_test['Fare'] <= 7.91, 'Fare'] = int(0)
df_test.loc[(df_test['Fare'] > 7.91) & (df_test['Fare'] <= 14.454), 'Fare'] = int(1)
df_test.loc[(df_test['Fare'] > 14.454) & (df_test['Fare'] <= 31), 'Fare']   = int(2)
df_test.loc[ df_test['Fare'] > 31, 'Fare'] = int(3)

In [34]:
# AGE
df_raw.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [35]:
df_raw.loc[ df_raw['Age'] <= 16, 'Age']  = int(0)
df_raw.loc[(df_raw['Age'] > 16) & (df_raw['Age'] <= 32), 'Age'] = int(1)
df_raw.loc[(df_raw['Age'] > 32) & (df_raw['Age'] <= 48), 'Age'] = int(2)
df_raw.loc[(df_raw['Age'] > 48) & (df_raw['Age'] <= 64), 'Age'] = int(3)
df_raw.loc[ df_raw['Age'] > 64, 'Age'] = int(4)

In [36]:
df_test.loc[ df_test['Age'] <= 16, 'Age']  = int(0)
df_test.loc[(df_test['Age'] > 16) & (df_test['Age'] <= 32), 'Age'] = int(1)
df_test.loc[(df_test['Age'] > 32) & (df_test['Age'] <= 48), 'Age'] = int(2)
df_test.loc[(df_test['Age'] > 48) & (df_test['Age'] <= 64), 'Age'] = int(3)
df_test.loc[ df_test['Age'] > 64, 'Age'] = int(4)

In [37]:
# Cabin
df_raw['Cabin_'] = df_raw['Cabin'].apply(lambda x: 0 if type(x) == float else x[0])
df_test['Cabin_'] = df_test['Cabin'].apply(lambda x: 0 if type(x) == float else x[0])

In [38]:
display_all(df_raw.tail().T)

,886,887,888,889,890
PassengerId,887,888,889,890,891
Survived,0,1,0,1,0
Pclass,2,1,3,1,3
Name,"Montvila, Rev. Juozas","Graham, Miss. Margaret Edith","Johnston, Miss. Catherine Helen ""Carrie""","Behr, Mr. Karl Howell","Dooley, Mr. Patrick"
Sex,male,female,female,male,male
Age,1,1,NaN,1,1
SibSp,0,0,1,0,0
Parch,0,0,2,0,0
Ticket,211536,112053,W./C. 6607,111369,370376
Fare,13,30,23.45,30,7.75


In [39]:
display_all(df_test.tail().T)

,413,414,415,416,417
PassengerId,1305,1306,1307,1308,1309
Pclass,3,1,3,3,3
Name,"Spector, Mr. Woolf","Oliva y Ocana, Dona. Fermina","Saether, Mr. Simon Sivertsen","Ware, Mr. Frederick","Peter, Master. Michael J"
Sex,male,female,male,male,male
Age,NaN,2,2,NaN,NaN
SibSp,0,0,0,0,1
Parch,0,0,0,0,1
Ticket,A.5. 3236,PC 17758,SOTON/O.Q. 3101262,359309,2668
Fare,0,0,0,0,0
Cabin,NaN,C105,NaN,NaN,NaN


In [40]:
col_drop = ["PassengerId", "Name", "Ticket", "Cabin", "SibSp"]

In [41]:
df_raw = df_raw.drop(col_drop , axis = 1)
df_test = df_test.drop(col_drop , axis = 1)

In [42]:
train_cats(df_raw)
apply_cats(df_test, df_raw)

In [43]:
df_test['Survived'] = np.repeat(0, df_test.shape[0])

In [44]:
type(df_raw)

pandas.core.frame.DataFrame

In [45]:
df, y, nas = proc_df(df_raw, 'Survived')
df_test_, _, nas = proc_df(df_test, 'Survived', na_dict=nas)

In [46]:
df.head(50)

,Pclass,Sex,Age,Parch,Fare,Embarked,Title,FamilySize,Cabin_,Age_na
0,3,2,1.0,0,7.2500,3,3,2,1,False
1,1,1,2.0,0,71.2833,1,4,2,4,False
2,3,1,1.0,0,7.9250,3,2,1,1,False
3,1,1,2.0,0,53.1000,3,4,2,4,False
4,3,2,2.0,0,8.0500,3,3,1,1,False
5,3,2,1.0,0,8.4583,2,3,1,1,True
6,1,2,3.0,0,51.8625,3,3,1,6,False
7,3,2,0.0,1,21.0750,3,1,5,1,False
8,3,1,1.0,2,11.1333,3,4,3,1,False
9,2,1,0.0,0,30.0708,1,4,2,1,False


In [47]:
nas

{'Age': 1.0, 'Fare': 0.0}

In [48]:
df_test_.head(100)

,Pclass,Sex,Age,Parch,Fare,Embarked,Title,FamilySize,Cabin_,Age_na,Fare_na
0,3,2,2.0,0,0.0,2,3,1,1,False,False
1,3,1,2.0,0,0.0,3,4,2,1,False,False
2,2,2,3.0,0,0.0,2,3,1,1,False,False
3,3,2,1.0,0,0.0,3,3,1,1,False,False
4,3,1,1.0,1,0.0,3,4,3,1,False,False
5,3,2,0.0,0,0.0,3,3,1,1,False,False
6,3,1,1.0,0,0.0,2,2,1,1,False,False
7,2,2,1.0,1,0.0,3,3,3,1,False,False
8,3,1,1.0,0,0.0,1,4,1,1,False,False
9,3,2,1.0,0,0.0,3,3,3,1,False,False


In [49]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def accuracy(x,y): return (x == y).mean()

def print_score(m, X_train, y_train, X_valid, y_valid):
    res = [accuracy(m.predict(X_train), y_train), accuracy(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [50]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 100  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((791, 10), (791,), (100, 10))

In [51]:
m = RandomForestClassifier(n_estimators=500, min_samples_leaf=15, n_jobs=-1, oob_score=False)
m.fit(X_train, y_train)
print_score(m, X_train, y_train, X_valid, y_valid)

[0.831858407079646, 0.82, 0.831858407079646, 0.82]


In [ ]:
#rslt = pd.DataFrame(data = {'PassengerId': df_test['PassengerId'], 'Survived': y_test_hat})

In [ ]:
#rslt.head()

In [ ]:
#rslt.to_csv('titanic.csv', index = False)
